# Skip-gram Word2Vec
In questo notebook, andremo ad utilizzare PyTorch per implementare un [algoritmo Word2Vec](https://en.wikipedia.org/wiki/Word2vec) usando l'architettura skip-gram. Implementando quessto impareremo come funziona il words embedding nel natural language processing. 


Qui abbiamo delle risorse che si possono consultare per capire meglio cosa andremo a fare. La lettura di questi documenti aiuta a capire bene questo documento.

* Una ottima [introduzione concettuale](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) del Word2Vec di Chris McCormick
* Il primo paper [Word2Vec](./docs/NIPS-2013-distributed-representations-of-words-and-phrases-and-their-compositionality-Paper.pdf) di Mikolov e altri.

---
## Word embeddings

Quando si ha a che fare con le parole nel testo, si finisce con una decina di migliaia di classi di parole di analizzare; una per ogni classe di parole del vocabolario. Cercare di codificare queste parole tramite vettore one-hot è estremamente inefficiente perchè molti valori nel vettore one-hot è impostata a zero. Dunque la moltiplicazione con la matrice che avviene tra il vettore one-hot e il primo hidden layer avrà molti valori impostati a zero.

<img src='images/one_hot_encoding.png' width=50%>

Per risolvere questo problema e aumentare l'efficienza delle nostre reti, andremo ad usare quello che chiamiamo **embeddings**.
Il livello di embeddings sostituisce le moltiplicazioni che la nostra rete dovrebbe fare con i vettori one-hot, concettualmente andiamo a togliere quelle moltiplicazioni e andiamo a recuperare direttamente i pesi che la rete avrebbe appreso.

<img src='images/lookup_matrix.png' width=50%>

Invece di eseguire la moltiplicazione della matrice, usiamo la matrice dei pesi come una tabella di lookup.
Codifichiamo le parole come interi, per esempio "heart" è codificato come 985, "mind" come 18094. Poi per ottenere il valore dell'hidden layer basta prendere il valore alla riga 958 della matrice di embedding. Questo processo è chiamato **embedding lookup** e il numero delle hidden unit è chiamato anche **embedding dimension**.

<img src='images/tokenize_lookup.png' width=50%>

Non c'è nulla di magico in quello che stiamo facendo. La tabella di lookup degli embeddings è solatemente una matrice di pesi e il lookup è solamente una scorciatoia per la moltiplicazione della matrice. La tabella di lookup è aggiornata come una matrice di pesi.

Gli embeddings non sono solo usati con le parole, ovviamente. Possiamo usarli su un qualsiasi modello che ha un numero elevato di classi. Un particolare tipo di modello è chiamato **Word2Vec** usa il layer di embedding per trovare una rappresentazione vettoriale che contiene un signficato semantico. 

---
## Word2Vec

L'algoritmo Word2Vec trova rappresentazioni molto effcienti di vettori che rappresentano una parola. Questi vettori contengono anche informazioni semantiche circa le parole.


<img src="images/context_drink.png" width=40%>

Le parole che compaiono in **contesti** simili ad esempio "coffee","tea" e "water" avranno vettori vicini l'uno all'altro.
Parole differenti saranno più lontane dalle altre e le relazioni potranno essere rappresentate dalla distanza nello spazio vettoriale.

<img src="images/vector_distance.png" width=40%>

come detto prima ci sono due architetture per implementare il Word2Vec

* CBOW (Continuous Bag-Of-Words) 
* Skip-gram

<img src="images/word2vec_architectures.png" width=60%>

In questa implementazione, useremo l'architettura **skip-gram** in quanto da risultati migliori della CBOW.
Al nostro modello daremo in pasto una parola e proveremo a predire le parole che la circondano nel testo.
In questo modo, possiamo eseguire l'addestramento della rete per ottenere rappresentazioni di parole che compaioni in contesti simili.

---
## Caricamento dei dati

Andiamo a caricare i dati nella cartella `data`.

In [1]:
import os
import time
import requests
import shutil
from tqdm.auto import tqdm

dataset = "https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip"
folder_result = "./data/" 

with requests.get(dataset, stream=True, verify=False) as r:
    total_length = int(r.headers.get("Content-Length"))
    with tqdm.wrapattr(r.raw, "read", total=total_length, desc="")as raw:
        with open(f"{os.path.join(folder_result,os.path.basename(r.url))}", 'wb') as output:
            shutil.copyfileobj(raw, output)

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


  0%|          | 0/31344016 [00:00<?, ?it/s]

In [2]:
import zipfile
with zipfile.ZipFile("./data/text8.zip","r") as zip_ref:
    zip_ref.extractall(folder_result)

visualizziamo ora il testo caricato

In [3]:
# read in the extracted text file      
with open('data/text8') as f:
    text = f.read()

# print out the first 100 characters
print(text[:100])

 anarchism originated as a term of abuse first used against early working class radicals including t


## Preprocessamento

Andiamo a sistemare il testo per facilitare la fase di trainig. Ci viene in aiuto il file `utils.py`.
La funzione `preprocess` fa queste poche cose:

* Converte la punteggiatura in tokens, dunque una virgola viene cambiata in ` <PERIOD> `. In questo dataset non c'è punteggiatura ma questo può aiutare in altri problemi di NLP.

* Rimuove tutte le parole che compaiono 5 o *meno* volte nel dataset. Questo andra a ridurre i problemi legati al rumore nei dati e aumenta la qualità della rappresentazione vettoriale.


* Restituisce una lista di parole del testo.

Questo prende qualche secondo per eseguire il calcolo, visto che il file è abbastanza larga.

Trasforma la riga seguente in codice in colab


!wget https://raw.githubusercontent.com/fdalforno/nlp/master/utils.py

In [4]:
import utils

# get list of words
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [5]:
# print some stats about this word data
print("Total words in text: {}".format(len(words)))
print("Unique words: {}".format(len(set(words)))) # `set` removes any duplicate words

Total words in text: 16680599
Unique words: 63641


### Dizionari
Ora, Andiamo a creare due dizionari per convertire le parole in interi e il contrario (da interi e parole). Questo viene eseguito ancora nel file `utils.py`. La funzione `create_lookup_tables` prende una lista di parole di un testo e crea due dizionari.

Gli interi sono assegnati in ordine decrescente di frequenza, dunque la parola più frequente ("the") prende il token 0, la parola prossima più frequente prende il codice 1 e così via.

Una volta che abbiamo ottenuto i nostri dizionari, le parole vengono convertite in interi e salvate nella lista `int_words`.

In [6]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

print(int_words[:30])

[5233, 3080, 11, 5, 194, 1, 3133, 45, 58, 155, 127, 741, 476, 10571, 133, 0, 27349, 1, 0, 102, 854, 2, 0, 15067, 58112, 1, 0, 150, 854, 3580]


## Subsampling

Le parole che compaiono spesso come "the", "of" e "for" non forniscono molto contesto alle parole vicine.
Se scartiamo alcune di esse, possiamo rimuovere un pò del rumore dai dati e come risultato otteniamo anche che la fase di training risulta più veloce con una migliore rappresentazione. Questo processo è chiamato da Mikolov *subsampling*.

Per ogni parola $w_i$ nel dataset di training, le scarteremo con la probabilità data da:

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

dove $t$ è un parametro di threshold e $f(w_i)$ è la frequenza della parola $w_i$ nel dataset.

$$ P(0) = 1 - \sqrt{\frac{1*10^{-5}}{1*10^6/16*10^6}} = 0.98735 $$

Implementiamo il subsampling per le parole `int_words`. 

In [7]:
from collections import Counter
import random
import numpy as np

threshold = 1e-5
word_counts = Counter(int_words)

In [8]:
print(list(word_counts.items())[3])  # dictionary of int_words, how many times they appear

(5, 325873)


In [9]:
total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}

In [10]:
# discard some frequent words, according to the subsampling equation
# create a new list of words for training
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

In [11]:
print(train_words[:30])

[5233, 3133, 10571, 27349, 102, 854, 15067, 58112, 3580, 10712, 3672, 708, 7088, 247, 44611, 5233, 8983, 4147, 6437, 4186, 362, 5233, 447, 1818, 6753, 7573, 1774, 566, 11064, 89]


## creare batches

Ora che i nostri dati sono ben sistemati, dobbiamo preparli per farli passare attraverso la nostra rete. Con l'architettura skip-gram , per ogni parola del testo, vogliamo definire un _contesto_ e cattura tutte le parole in una finestra attorno alla parola di dimensione $C$. 

Da [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf):

"Visto che le parole più distanti sono solitamente meno collegate alla parola corrente che alle parole più vicine, diamo meno peso alle parole distanti campionando meno frequentemente queste parole nei dati di training. Se scegliamo ad esempio $C = 5$, per ogni parola di trainig sceglieremo un numero random $R$ nel range $[ 1: C ]$, e poi useremo le $R$ parole a sinistra e a destra della parola corrente come etichetta corretta."

Implementiamo la funzione `get_target`  ad esempio se abbiamo la seguente lista prendiamo idx=2, `741`: 

```
[5233, 58, 741, 10571, 27349, 0, 15067, 58112, 3580, 58, 10712]
```

Per `R=2`, `get_target` ritorna questi quattro valori:
```
[5233, 58, 10571, 27349]
```

In [12]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = words[start:idx] + words[idx+1:stop+1]
    
    return list(target_words)

In [13]:
# test your code!

# run this cell multiple times to check for random window selection
int_text = [i for i in range(10)]
print('Input: ', int_text)
idx=5 # word index of interest

target = get_target(int_text, idx=idx, window_size=5)
print('Target: ', target)  # you should get some indices around the idx

Input:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Target:  [2, 3, 4, 6, 7, 8]


### Generazione dei batch

Qui abbiamo un generatore che ritorna dei batch con dati di input e target per il nostro modello usando la funzione `get_target`
costruita sopra.

L'idea è quella di costruire un insieme di parole di dimensione `batch_size` da una lista di parole.
Poi per ogniuno di questi batch si ottiene la parola target in una finestra.

In [14]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

In [15]:
int_text = [i for i in range(20)]
x,y = next(get_batches(int_text, batch_size=4, window_size=5))

print('x\n', x)
print('y\n', y)

x
 [0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3]
y
 [1, 2, 3, 0, 2, 0, 1, 3, 0, 1, 2]


## Costruzione del grafo

Qui sotto una approssimazione del diagramma della struttura della nostra rete. 

<img src="./images/skip_gram_arch.png" width=60%>

* le parole sono passate come batch di token codificati come interi 
* queste attraversano un hidden layer il nostro embedding layer.
* alla fine usiamo il layer di output con la parte softmax

Useremo il layer softmax per eseguire delle previsioni circa la parola del contesto come definito.
L'idea è quella di addestrare la nostra rete per trovare una rappresentazione efficiente delle nostre parole.

Possiamo dimenticarci dell'ultimo layer softmax non ci importa molto di eseguire delle previsioni con questa rete.
Volgiamo soltanto usare la matrice di embedding che verrà usata in _altre_ reti neuronali.

---

## Validazione
Qui stiamo creando una funzione che ci aiuterà ad osservare cosa il nostro modello sta imparando.
Dobbiamo scegliere alcune parole comuni e alcune parole non comuni.


Poi mostreremo le parole più vicine alle parole selezionate usando il coseno di similitudine:

<img src="./images/two_vectors.png" width=30%>

$$
\mathrm{similarity} = \cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|}
$$


Possiamo codificare le parole di validazione come vettori $\vec{a}$ usando la tebella di embedding, poi calcoliamo la similarità per ogni word vector $\vec{b}$  nella tabella di embedding.

Con queste similitudini possiamo mostrare le parole di validazione e le parole nella tabella di embedding semanticamente simili alla stessa. Questo è un modo carino per controllare se l'operazione di embedding sta raggruppando parole con un significato semantico simile.

In [16]:
def cosine_similarity(embedding, valid_size=16, valid_window=100, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the embedding matrix.
        Here, embedding should be a PyTorch embedding module.
    """
    
    # Here we're calculating the cosine similarity between some random words and 
    # our embedding vectors. With the similarities, we can look at what words are
    # close to our random words.
    
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # pick N words from our ranges (0,window) and (1000,1000+window). lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

## SkipGram model


Definiamo il modello SkipGram.
> Abbiamo bisgono di definire un [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) e nella parte finale un layer softmax.

Il layer di Embedding vuole i seguenti parametri:
* **num_embeddings** - la dimensione del dizionario di embeddings, o quante righe vogliamo nella nostra matrice di embedding.
* **embedding_dim** - la dimensione di ogni vettore; ovvero la dimensione di embedding

In [17]:
import torch
from torch import nn
import torch.optim as optim

In [18]:
class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        
        self.embed = nn.Embedding(n_vocab, n_embed)
        self.output = nn.Linear(n_embed, n_vocab)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.embed(x)
        scores = self.output(x)
        log_ps = self.log_softmax(scores)
        
        return log_ps

## Training



In [19]:
# check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_dim=300 # you can change, if you want

model = SkipGram(len(vocab_to_int), embedding_dim).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

print_every = 500
steps = 0
epochs = 5

# train for some number of epochs
for e in range(epochs):
    
    # get input and target batches
    for inputs, targets in get_batches(train_words, 512):
        steps += 1
        inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
        inputs, targets = inputs.to(device), targets.to(device)
        
        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if steps % print_every == 0:                  
            # getting examples and similarities      
            valid_examples, valid_similarities = cosine_similarity(model.embed, device=device)
            _, closest_idxs = valid_similarities.topk(6) # topk highest similarities
            
            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...")

KeyboardInterrupt: 

## Visualizzare i word vector

Useremo il metodo T-SNE per visualizzare come i nostri word vector in alta dimensionalità si raggreuppano assieme.
T-SNE viene usato per proiettare questi vettori in uno spazio bidimensionale preservando le strutture locale. 

Per avere più informazioni circa questo metodo possiamo documentarci leggendo questo [post di Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/).

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
# getting embeddings from the embedding layer of our model, by name
embeddings = model.embed.weight.to('cpu').data.numpy()

In [ ]:
viz_words = 600
tsne = TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)